# Recipe2Cuisine

In [1]:
import pandas as pd

In [2]:
recipes_df = pd.read_json('./recipies.json',orient='columns')

In [3]:
recipes_df[['ingredients','cuisine']]

,ingredients,cuisine
0,"[romaine lettuce, black olives, grape tomatoes...",greek
1,"[plain flour, ground pepper, salt, tomatoes, g...",southern_us
2,"[eggs, pepper, salt, mayonaise, cooking oil, g...",filipino
3,"[water, vegetable oil, wheat, salt]",indian
4,"[black pepper, shallots, cornflour, cayenne pe...",indian
5,"[plain flour, sugar, butter, eggs, fresh ginge...",jamaican
6,"[olive oil, salt, medium shrimp, pepper, garli...",spanish
7,"[sugar, pistachio nuts, white almond bark, flo...",italian
8,"[olive oil, purple onion, fresh pineapple, por...",mexican
9,"[chopped tomatoes, fresh basil, garlic, extra-...",italian


In [4]:
recipes_df['ingredients_string'] = recipes_df.ingredients.str.join(' ')

In [5]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/peterschnatz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
ingred_list = recipes_df.ingredients_string.str.cat(sep=' ')
#function to split text into word
tokens = word_tokenize(ingred_list)
vocabulary = set(tokens)
print(len(vocabulary))
frequency_dist = nltk.FreqDist(tokens)
sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]

3554


['pepper',
 'salt',
 'oil',
 'garlic',
 'ground',
 'fresh',
 'sauce',
 'sugar',
 'onions',
 'cheese',
 'chicken',
 'olive',
 'black',
 'water',
 'red',
 'flour',
 'tomatoes',
 'butter',
 'green',
 'powder',
 'chopped',
 'cloves',
 'juice',
 'white',
 'onion',
 'eggs',
 'rice',
 'cream',
 'cilantro',
 'milk',
 'lemon',
 'vegetable',
 'leaves',
 'large',
 'ginger',
 'corn',
 'dried',
 'vinegar',
 'lime',
 'soy',
 'cumin',
 'all-purpose',
 'broth',
 'wine',
 'chili',
 'bell',
 'parsley',
 'sesame',
 'beans',
 'grated']

In [10]:
sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[-50:]

['sato',
 'Jagermeister',
 'huckleberries',
 'crosswise',
 'pike',
 'matzos',
 'nettle',
 'bellpepper',
 'Tandoori',
 'Naan',
 'sucrolose',
 'Holland',
 'House',
 'Wine',
 'frankfurters',
 'soy-based',
 'tortelloni',
 'Potato',
 'Matzo',
 'Meal',
 'arctic',
 'tonic',
 'Chartreuse',
 'aloe',
 'Honeysuckle',
 'White®',
 'Turkey',
 'cumberland',
 'Colman',
 'manouri',
 'ribeye',
 'riblets',
 'Plus',
 'conchiglie',
 'Makers',
 'Mark',
 'Whisky',
 'ranch-style',
 'ketjap',
 'Daiya',
 'Slim',
 'Cotto',
 'Salami',
 'Challenge',
 'Brew',
 'Size',
 'Bags',
 'Dip',
 'lop',
 'chong']

In [11]:
print(recipes_df.cuisine.nunique())
print(recipes_df.cuisine.value_counts())
print(recipes_df.cuisine.value_counts().max() / len(recipes_df))

20
italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64
0.19706340825665009


If I blindly just assign all of the recipes to italian, I will get 20% accuracy. What if I randomly select a cuisine and assign it?

In [12]:
import random

In [13]:
from  sklearn.metrics  import accuracy_score
cuisine_list = list(recipes_df.cuisine.unique())
predict_cuisine_random = [random.choice(cuisine_list) for i in range(len(recipes_df))]
print(accuracy_score(recipes_df.cuisine,predict_cuisine_random))

0.051968622718358726


In [14]:
from itertools import permutations 
import numpy as np
from sklearn.metrics import f1_score, classification_report

In [15]:
acc_scores = list()
for i in range(1000):
    cuisines_perm = np.random.permutation(list(recipes_df.cuisine))
    acc_scores.append(accuracy_score(recipes_df.cuisine,cuisines_perm))

print('Average accuracy for benchmark: %.2f' %(sum(acc_scores)/len(acc_scores)))
print(classification_report(recipes_df.cuisine,cuisines_perm))

Average accuracy for benchmark: 0.10
              precision    recall  f1-score   support

   brazilian       0.01      0.01      0.01       467
     british       0.02      0.02      0.02       804
cajun_creole       0.03      0.03      0.03      1546
     chinese       0.08      0.08      0.08      2673
    filipino       0.02      0.02      0.02       755
      french       0.07      0.07      0.07      2646
       greek       0.03      0.03      0.03      1175
      indian       0.08      0.08      0.08      3003
       irish       0.01      0.01      0.01       667
     italian       0.20      0.20      0.20      7838
    jamaican       0.02      0.02      0.02       526
    japanese       0.04      0.04      0.04      1423
      korean       0.03      0.03      0.03       830
     mexican       0.15      0.15      0.15      6438
    moroccan       0.02      0.02      0.02       821
     russian       0.02      0.02      0.02       489
 southern_us       0.11      0.11      0.11 

In [16]:
list(recipes_df.cuisine.unique())

['greek',
 'southern_us',
 'filipino',
 'indian',
 'jamaican',
 'spanish',
 'italian',
 'mexican',
 'chinese',
 'british',
 'thai',
 'vietnamese',
 'cajun_creole',
 'brazilian',
 'french',
 'japanese',
 'irish',
 'korean',
 'moroccan',
 'russian']

Excellent! So it does even worse!!

## Straight out of the box TF-IDF Naive-Bayes

In [17]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/peterschnatz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
stop_words = set(stopwords.words('english'))
tokens = [w for w in tokens if not w in stop_words]

In [19]:
vocab_no_stops = set(tokens)
len(vocab_no_stops)

3530

In [20]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,8))
wordcloud = WordCloud().generate_from_frequencies(frequency_dist)
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

fig.savefig('WordCloud.png')

<Figure size 1500x800 with 1 Axes>

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(recipes_df.ingredients_string, recipes_df.cuisine, test_size=0.2,
                                                   random_state=123)

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

(31819, 2894) (7955, 2894)


In [25]:
train_vectors

<31819x2894 sparse matrix of type '<class 'numpy.float64'>'
	with 609826 stored elements in Compressed Sparse Row format>

In [26]:
from sklearn.naive_bayes import MultinomialNB

In [27]:
clf = MultinomialNB().fit(train_vectors, y_train)

In [28]:
from  sklearn.metrics  import accuracy_score
predicted = clf.predict(test_vectors)
print(accuracy_score(y_test,predicted))

0.6696417347580138


In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
clf = RandomForestClassifier(n_estimators=50).fit(train_vectors, y_train)

In [29]:
from  sklearn.metrics  import accuracy_score
predicted = clf.predict(test_vectors)
print(accuracy_score(y_test,predicted))
print(classification_report(y_test,predicted))

0.7426775612822124
              precision    recall  f1-score   support

   brazilian       0.84      0.42      0.56        90
     british       0.60      0.22      0.33       157
cajun_creole       0.82      0.66      0.73       313
     chinese       0.70      0.88      0.78       518
    filipino       0.74      0.51      0.60       158
      french       0.58      0.51      0.54       567
       greek       0.85      0.65      0.74       222
      indian       0.82      0.91      0.87       597
       irish       0.64      0.26      0.37       131
     italian       0.70      0.91      0.79      1501
    jamaican       0.90      0.44      0.59       101
    japanese       0.87      0.65      0.74       293
      korean       0.87      0.61      0.72       187
     mexican       0.84      0.93      0.88      1309
    moroccan       0.93      0.58      0.71       170
     russian       0.84      0.21      0.33       102
 southern_us       0.63      0.77      0.69       874
     spa

That's a pretty good improvement over benchmark!

Let's try to spruce it up a bit though. Let's see if we can deal with different ingredients meaning essentially the same thing.

Preprocess the ingredient lists to make sure there are no numbers or non-alphabet characters and everything is lower case. Removing the numbers, i.e. the amount of each ingredient might lose a bit of information. But since the amount of each ingredient is typically not said, the information it gives is minimal.

In [30]:
list(set(recipes_df.ingredients[0]))

['romaine lettuce',
 'feta cheese crumbles',
 'garlic',
 'seasoning',
 'garbanzo beans',
 'black olives',
 'purple onion',
 'grape tomatoes',
 'pepper']

Take out non-alphabet characters, keep spaces and make everything lowercase

In [32]:
ingred_lists_no_repeats = list()

import re

regex = re.compile('[^a-zA-Z_]')
#First parameter is the replacement, second parameter is your input string
regex.sub('', 'ab3d*E')
#Out: 'abdE'

for i, row in enumerate(recipes_df['ingredients']):
    ingred_list_processed = list()
    
    for term in row:
        temp = regex.sub('',term).lower()
        ingred_list_processed.append(temp)

    ingred_no_repeats = list(set(ingred_list_processed))
    ingred_lists_no_repeats.append(ingred_no_repeats)

In [ ]:
recipes_df['ingred_processed'] = ingred_lists_no_repeats

Let's see if there are repeats ingredient lists which are applied to more than one cuisine

In [ ]:
ingred_sets = [set(row) for row in list(recipes_df.ingred_processed)]

In [ ]:
recipes_df['ingred_set'] = ingred_sets

In [ ]:
repeat_ingred_list = recipes_df.ingred_set.value_counts()[recipes_df.ingred_set.value_counts() > 1]

In [ ]:
len(repeat_ingred_list)

In [ ]:
for el in repeat_ingred_list.index:
    cuisine_set = set(recipes_df.loc[recipes_df.ingred_set == el, 'cuisine'])
    if (len(cuisine_set) != 1):
        print(el)
        print(str(cuisine_set)+'\n')
        

In [ ]:
recipes_df.columns

Find ingredients containing "tomat"

In [ ]:
tomat_terms = list()
for row in recipes_df['ingred_processed']:
#     print(row)
    term = [ingredient for ingredient in row if 'tomat' in ingredient]
    tomat_terms += term

tomat_series = pd.Series(tomat_terms).value_counts()[:50]
# set(tomat_terms)
# words = pd.DataFrame(rows, columns=['book', 'word'])
# words.head()

In [ ]:
recipes_df['ingred_proc_string'] = recipes_df.ingred_processed.str.join(' ')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(recipes_df.ingred_proc_string, recipes_df.cuisine, test_size=0.2,
                                                   random_state=123)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
clf = MultinomialNB().fit(train_vectors, y_train)

In [ ]:
from  sklearn.metrics  import accuracy_score
y_pred_train = clf.predict(train_vectors)
y_pred = clf.predict(test_vectors)

print(accuracy_score(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))

print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

All that processing - getting rid of duplicates in the ingredients lists did NOTHING!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=50).fit(train_vectors, y_train)

In [ ]:
from  sklearn.metrics  import accuracy_score
y_pred_train = clf.predict(train_vectors)
y_pred = clf.predict(test_vectors)

print(accuracy_score(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))

print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

## Stemming!

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [ ]:
recipes_df

In [ ]:
porter = PorterStemmer()
ing_port_list = list()
i=0

for row in recipes_df['ingred_proc_string']:
    ing_port = porter.stem(row)
#     ing_port_list += ing_port
    ing_port_list.append(ing_port)
    

recipes_df['ing_proc_str_port'] = ing_port_list

In [ ]:
recipes_df.columns

In [ ]:
recipes_df.drop(columns='ingredients',axis=1,inplace=True)

In [ ]:
recipes_df.drop(columns='ingredients_string',axis=1,inplace=True)

In [ ]:
porter = PorterStemmer()
porter_list = list()
i=0

for i, row in enumerate(recipes_df['ingred_processed']):
    ing_port_list = list()
    for term in row:
        ing_port = porter.stem(term)
        ing_port_list.append(ing_port)
    porter_list.append(ing_port_list)


print(porter_list)
recipes_df['ing_proc_lst_port'] = porter_list

In [ ]:
recipes_df.ing_proc_lst_port

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
tomat_series

In [ ]:
from fuzzywuzzy import process
str2Match = tomat_series.index[0]
strOptions = list(tomat_series.index)
Ratios = process.extract(str2Match,strOptions,limit=50,scorer=fuzz.token_set_ratio)
print(Ratios)
# You can also select the string with the highest matching percentage
highest = process.extractOne(str2Match,strOptions)
print(highest)

In [ ]:
recipes_df.ing_proc_str_port

In [ ]:
recipes_df.columns

In [ ]:
recipes_df.columns

In [ ]:
recipes_df.ing_proc_lst_port[0]

In [ ]:
fuzz.token_sort_ratio(recipes_df.ing_proc_lst_port[0][1],"tomatoes")

In [ ]:
tomat_series.index

In [ ]:
process.extract("tomato",tomat_series.index,scorer=fuzz.token_sort_ratio)

In [ ]:
recipes_df.ingred_processed

In [ ]:
def get_ratio(row):
    name = row['ingred_processed']
    return process.extractOne("tomato",name,scorer=fuzz.token_sort_ratio)[1]

recipes_df[recipes_df.apply(get_ratio, axis=1) > 70]

In [ ]:
tomat_terms = list()
for row in recipes_df['ing_proc_lst_port']:
#     print(row)
    term = [ingredient for ingredient in row if 'tomat' in ingredient]
    tomat_terms += term

tomat_series = pd.Series(tomat_terms).value_counts()[:50]

In [ ]:
tomat_series

In [ ]:
tomat_series.index

Tomat_series pre-porter

In [ ]:
tomat_terms = list()
for row in recipes_df['ingred_processed']:
#     print(row)
    term = [ingredient for ingredient in row if 'tomat' in ingredient]
    tomat_terms += term

tomat_series = pd.Series(tomat_terms).value_counts()[:50]

In [ ]:
process.extract("tomatoes",tomat_series.index,scorer=fuzz.token_set_ratio,limit=50)

In [ ]:
def get_ratio(row):
    token_scores = list()
    name = row['ingred_processed']
    for ingredient in name:
        token_scores.append(fuzz.token_sort_ratio(ingredient,"tomatoes"))

    return token_scores

for index, row in recipes_df.iterrows():
    scores = get_ratio(row)
    for score in scores:
        if (score > 70):
            row['ingred_processed'][scores.index(score)] = "tomatoes"

In [ ]:
recipes_df.ingred_processed

In [ ]:
recipes_df.ingred_processed

In [ ]:
recipes_df['new_string'] = recipes_df.ingred_processed.str.join(' ')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(recipes_df.new_string, recipes_df.cuisine, test_size=0.2,
                                                   random_state=123)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

In [ ]:
parameters={'max_depth':max_depth, 'min_samples_leaf':min_samples_leaf}
rf = RandomForestRegressor(n_estimators=20,max_features='sqrt')
rf = GridSearchCV(rf, parameters,cv=3)
clf = rf.fit(train_vectors, y_train)

In [ ]:
from  sklearn.metrics  import accuracy_score
from sklearn.metrics import f1_score, classification_report
predicted = clf.predict(test_vectors)
print(accuracy_score(y_test,predicted))
print(classification_report(y_test,predicted))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

def randomforest(max_depth,min_samples_leaf,n_estimators):
    parameters={'max_depth':max_depth, 'min_samples_leaf':min_samples_leaf}
    
    vectorizer = TfidfVectorizer()
    train_vectors = vectorizer.fit_transform(X_train)
    test_vectors = vectorizer.transform(X_test)
    
    # Create and fit random forest
    clf = RandomForestClassifier(n_estimators=30,max_features='sqrt')
    clf = GridSearchCV(clf, parameters,cv=3)
    clf.fit(train_vectors,y_train)

    scores = clf.cv_results_['mean_test_score'].reshape(len(max_depth),len(min_samples_leaf))

    # Plot heatmap of R^2 values
    plt.figure(figsize=(5,5))
    plt.imshow(scores,cmap=plt.cm.inferno)
    plt.xlabel('min_samples_leaf')
    plt.ylabel('max_depth')
    plt.xticks(np.arange(len(min_samples_leaf)),min_samples_leaf)
    plt.yticks(np.arange(len(max_depth)),max_depth)
    plt.colorbar()

    for i in range(len(max_depth)):
        for j in range(len(min_samples_leaf)):
            text = plt.text(j, i, scores[i, j].round(3),
                           horizontalalignment="center",
                           verticalalignment="center",
                           color="g")

    plt.title('Grid Search Results (R^2 score)')
    plt.show()

    print('Best parameters: ',clf.best_params_)

    y_pred_train = clf.predict(train_vectors)
    y_pred = clf.predict(test_vectors)

    print(accuracy_score(y_train,y_pred_train))
    print(classification_report(y_train,y_pred_train))
    
    print(accuracy_score(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    
    print('\nFeature Importances')
#     importance_frame = pd.DataFrame(list(zip(train_x.columns,clf.best_estimator_.feature_importances_)), columns=['feature','importance'])
#     print(importance_frame)
    
    return clf
    

In [ ]:
max_depth = np.floor(np.linspace(4,14,num=4))
min_samples_leaf = list(map(int,np.floor(np.logspace(1.0,3.0,num=4))))
n_estimators = 30

clf_model = randomforest(max_depth,min_samples_leaf,n_estimators)

In [ ]:
max_depth = np.floor(np.linspace(12,20,num=4))
min_samples_leaf = list(map(int,np.floor(np.logspace(0.2,1.2,num=4))))
n_estimators = 30

clf_model = randomforest(max_depth,min_samples_leaf,n_estimators)

In [ ]:
rf.score

In [ ]:
import seaborn as sns

In [ ]:
fig = plt.figure(figsize=(15,8))
sns.countplot(recipes_df.cuisine)
plt.xticks(rotation=45, ha='right')

plt.rc('axes', labelsize=24)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=24)    # fontsize of the tick labels
plt.rc('ytick', labelsize=24) 
plt.tight_layout()
fig.savefig('CuisineCount.png', transparent=True)

In [ ]:
flour_terms = list()
for row in recipes_df['ingred_processed']:
#     print(row)
    term = [ingredient for ingredient in row if 'flour' in ingredient]
    flour_terms += term

flour_series = pd.Series(flour_terms).value_counts()[:50]

In [ ]:
process.extract("flour",flour_series.index,scorer=fuzz.token_sort_ratio,limit=50)

In [ ]:
def get_ratio(row):
    token_scores = list()
    name = row['ingred_processed']
    for ingredient in name:
        token_scores.append(fuzz.token_sort_ratio(ingredient,"flour"))

    return token_scores

for index, row in recipes_df.iterrows():
    scores = get_ratio(row)
    if (max(scores) > 60):
        row['ingred_processed'][scores.index(max(scores))] = "flour"

In [ ]:
token_scores.index(max(token_scores))

In [ ]:
recipes_df.ingred_processed

In [ ]:
row = recipes_df.iloc[0]

token_scores = list()
name = row['ingred_processed']
for ingredient in name:
    token_scores.append(fuzz.token_sort_ratio(ingredient,"tomatoes"))
print(token_scores)
if (max(token_scores) > 70):
     row['ingred_processed'][token_scores.index(max(token_scores))] = "tomatoes"

In [ ]:
fuzz.token_sort_ratio("tomatoes","tomatoes")

In [ ]:
[73, 30, 100, 14, 24, 40, 14, 18, 21].index(max([73, 30, 100, 14, 24, 40, 14, 18, 21]))

In [ ]:
token_scores.index(max(token_scores))

In [ ]:
row['ingred_processed'][token_scores.index(max(token_scores))]

# Upsampling minority classes

In [ ]:
recipes_df.columns

In [ ]:
max(recipes_df.cuisine.value_counts())

In [ ]:
recipes_df.cuisine.value_counts()[0]

In [ ]:
# animal list
animal = ['cat', 'dog', 'rabbit', 'guinea pig']

# 'rabbit' element is removed
animal.remove('rabbit')

#Updated Animal List
print('Updated animal list: ', animal)

In [ ]:
old = ['p','m','l']
old.remove('l')
print(old)

In [ ]:
# Get list of cuisines
cuisine_lst = list(recipes_df.cuisine)
max_class = recipes_df.cuisine.value_counts().index[0]
max_counts = recipes_df.cuisine.value_counts()[0]

cuisine_lst.remove(max_class)

i_max_class = np.where(recipes_df.cuisine == max_class)[0]

for cuisine in cuisine_lst:
    # For every observation in class 1, randomly sample from class 0 with replacement
    i_cuisine = np.where(recipes_df.cuisine == cuisine)[0]
    i_upsamp_cuisine = np.random.choice(i_cuisine, size=max_counts, replace=True)

    # Join together class 0's upsampled target vector with class 1's target vector
    check = np.concatenate((X_train[i_upsamp_cuisine], X_train[i_max_class]))
    break

X_train[i_upsamp_cuisine]

In [ ]:
 X_train[i_max_class]

In [ ]:
check

In [ ]:
from sklearn.utils import resample

# Get list of cuisines
cuisine_lst = list(recipes_df.cuisine.unique())
max_class = recipes_df.cuisine.value_counts().index[0]
max_counts = recipes_df.cuisine.value_counts()[0]

cuisine_lst.remove(max_class)

i_max_class = np.where(recipes_df.cuisine == max_class)[0]

# Separate majority and minority classes
df_upsampled = recipes_df.iloc[i_max_class]
for cuisine in cuisine_lst: 
    # Upsample minority class
    i_cuisine = np.where(recipes_df.cuisine == cuisine)[0]
    df_minority_upsampled = resample(recipes_df.iloc[i_cuisine], 
                                     replace=True,     # sample with replacement
                                     n_samples=max_counts,    # to match majority class
                                     random_state=123) # reproducible results

    # Combine majority class with upsampled minority class
    df_upsampled = pd.concat([df_upsampled, df_minority_upsampled])
    print(cuisine)
    
# Display new class counts
df_upsampled.cuisine.value_counts()

In [ ]:
fig = plt.figure(figsize=(15,8))
sns.countplot(df_upsampled.cuisine)
plt.xticks(rotation=45, ha='right')

plt.rc('axes', labelsize=24)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=24)    # fontsize of the tick labels
plt.rc('ytick', labelsize=24) 
plt.tight_layout()
fig.savefig('CuisineCount_upsampled.png', transparent=True)

In [ ]:
X_train.unique()

In [ ]:
result = pd.concat([X_train, y_train], axis=1, sort=False)

In [ ]:
result.shape

In [ ]:
from sklearn.utils import resample

# Get list of cuisines
cuisine_lst = list(result.cuisine.unique())
max_class = result.cuisine.value_counts().index[0]
max_counts = result.cuisine.value_counts()[0]

cuisine_lst.remove(max_class)

i_max_class = np.where(result.cuisine == max_class)[0]

# Separate majority and minority classes
df_upsampled = result.iloc[i_max_class]
for cuisine in cuisine_lst: 
    # Upsample minority class
    i_cuisine = np.where(result.cuisine == cuisine)[0]
    df_minority_upsampled = resample(result.iloc[i_cuisine], 
                                     replace=True,     # sample with replacement
                                     n_samples=max_counts,    # to match majority class
                                     random_state=123) # reproducible results

    # Combine majority class with upsampled minority class
    df_upsampled = pd.concat([df_upsampled, df_minority_upsampled])
    print(cuisine)
    
# Display new class counts
df_upsampled.cuisine.value_counts()

In [ ]:
df_upsampled.columns

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

def randomforest(max_depth,min_samples_leaf,n_estimators):
    parameters={'max_depth':max_depth, 'min_samples_leaf':min_samples_leaf}
    
    vectorizer = TfidfVectorizer()
    train_vectors = vectorizer.fit_transform(df_upsampled.ingred_proc_string)
    test_vectors = vectorizer.transform(X_test)
    
    # Create and fit random forest
    clf = RandomForestClassifier(n_estimators=n_estimators,max_features='sqrt')
    clf = GridSearchCV(clf, parameters,cv=3)
    clf.fit(train_vectors,df_upsampled.cuisine)

    scores = clf.cv_results_['mean_test_score'].reshape(len(max_depth),len(min_samples_leaf))

    # Plot heatmap of R^2 values
    plt.figure(figsize=(5,5))
    plt.imshow(scores,cmap=plt.cm.inferno)
    plt.xlabel('min_samples_leaf')
    plt.ylabel('max_depth')
    plt.xticks(np.arange(len(min_samples_leaf)),min_samples_leaf)
    plt.yticks(np.arange(len(max_depth)),max_depth)
    plt.colorbar()

    for i in range(len(max_depth)):
        for j in range(len(min_samples_leaf)):
            text = plt.text(j, i, scores[i, j].round(3),
                           horizontalalignment="center",
                           verticalalignment="center",
                           color="g")

    plt.title('Grid Search Results (R^2 score)')
    plt.show()

    print('Best parameters: ',clf.best_params_)

    y_pred_train = clf.predict(train_vectors)
    y_pred = clf.predict(test_vectors)

    print(accuracy_score(df_upsampled.cuisine,y_pred_train))
    print(classification_report(df_upsampled.cuisine,y_pred_train))
    
    print(accuracy_score(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    
    print('\nFeature Importances')
#     importance_frame = pd.DataFrame(list(zip(train_x.columns,clf.best_estimator_.feature_importances_)), columns=['feature','importance'])
#     print(importance_frame)
    
    return clf
    

In [ ]:
max_depth = np.floor(np.linspace(4,14,num=4))
min_samples_leaf = list(map(int,np.floor(np.logspace(1.0,3.0,num=4))))
n_estimators = 50

clf_model = randomforest(max_depth,min_samples_leaf,n_estimators)

# Try upsampling on unprocessed ingredient lists

In [ ]:
recipes_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(recipes_df.ingredients_string, recipes_df.cuisine, test_size=0.2,
                                                   random_state=123)

In [ ]:
from sklearn.utils import resample

result = pd.concat([X_train, y_train], axis=1, sort=False)

# Get list of cuisines
cuisine_lst = list(result.cuisine.unique())
max_class = result.cuisine.value_counts().index[0]
max_counts = result.cuisine.value_counts()[0]

cuisine_lst.remove(max_class)

i_max_class = np.where(result.cuisine == max_class)[0]

# Separate majority and minority classes
df_upsampled = result.iloc[i_max_class]
for cuisine in cuisine_lst: 
    # Upsample minority class
    i_cuisine = np.where(result.cuisine == cuisine)[0]
    df_minority_upsampled = resample(result.iloc[i_cuisine], 
                                     replace=True,     # sample with replacement
                                     n_samples=max_counts,    # to match majority class
                                     random_state=123) # reproducible results

    # Combine majority class with upsampled minority class
    df_upsampled = pd.concat([df_upsampled, df_minority_upsampled])
    print(cuisine)
    
# Display new class counts
df_upsampled.cuisine.value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

def randomforest(max_depth,min_samples_leaf,n_estimators):
    parameters={'max_depth':max_depth, 'min_samples_leaf':min_samples_leaf}
    
    vectorizer = TfidfVectorizer()
    train_vectors = vectorizer.fit_transform(df_upsampled.ingredients_string)
    test_vectors = vectorizer.transform(X_test)
    
    # Create and fit random forest
    clf = RandomForestClassifier(n_estimators=n_estimators,max_features='sqrt')
    clf = GridSearchCV(clf, parameters,cv=3)
    clf.fit(train_vectors,df_upsampled.cuisine)

    scores = clf.cv_results_['mean_test_score'].reshape(len(max_depth),len(min_samples_leaf))

    # Plot heatmap of R^2 values
    plt.figure(figsize=(5,5))
    plt.imshow(scores,cmap=plt.cm.inferno)
    plt.xlabel('min_samples_leaf')
    plt.ylabel('max_depth')
    plt.xticks(np.arange(len(min_samples_leaf)),min_samples_leaf)
    plt.yticks(np.arange(len(max_depth)),max_depth)
    plt.colorbar()

    for i in range(len(max_depth)):
        for j in range(len(min_samples_leaf)):
            text = plt.text(j, i, scores[i, j].round(3),
                           horizontalalignment="center",
                           verticalalignment="center",
                           color="g")

    plt.title('Grid Search Results (R^2 score)')
    plt.show()

    print('Best parameters: ',clf.best_params_)

    y_pred_train = clf.predict(train_vectors)
    y_pred = clf.predict(test_vectors)

    print(accuracy_score(df_upsampled.cuisine,y_pred_train))
    print(classification_report(df_upsampled.cuisine,y_pred_train))
    
    print(accuracy_score(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    
    print('\nFeature Importances')
#     importance_frame = pd.DataFrame(list(zip(train_x.columns,clf.best_estimator_.feature_importances_)), columns=['feature','importance'])
#     print(importance_frame)
    
    return clf
    

In [ ]:
max_depth = np.floor(np.linspace(4,14,num=4))
min_samples_leaf = list(map(int,np.floor(np.logspace(1.0,3.0,num=4))))
n_estimators = 20

clf_model = randomforest(max_depth,min_samples_leaf,n_estimators)

In [ ]:
clf_model.cv_results_

In [ ]:
recipes_df.columns

In [ ]:
from collections import Counter

for cuisine in list(recipes_df.cuisine.unique()):

    words = list(recipes_df.loc[recipes_df.cuisine == cuisine, 'ingredients'])
    counter = Counter(words[0])
    for i in words[1:]: 
        counter.update(i)

    print(cuisine)
    print(counter.most_common(5))